In [1]:
%pip install erddapy

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 73.5/73.5 kB 1.8 MB/s eta 0:00:00:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.8/78.8 kB 11.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 113.4/113.4 kB 13.2 MB/s eta 0:00:00
  Attempting uninstall: h11
    Found existing installation: h11 0.14.0
    Uninstalling h11-0.14.0:
      Successfully uninstalled h11-0.14.0

[notice] A new release of pip is available: 24.0 -> 25.3
[notice] To update, run: pip3 install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [12]:
import pandas as pd
from erddapy import ERDDAP

erddap_obj = ERDDAP(
    server='https://coastwatch.noaa.gov/erddap',
    protocol='griddap',
    response='nc'  # Download as NetCDF (compatible with Xarray)
)

Like the issue with AQS versus AirNow, NOAA CoastWatch data includes versions that feature near real-time data and versions that are delayed for quality control purposes.

We'll use a version that is quality controlled for scientific research here, but it is possible to access realtime data if you are interested in fetching a dataset that is related to a current event of interest by selecting a different DATASET_ID. Note that those data have not gone through the same rigor of validation processes. 

NOTE If you need data older than 2016, you should look for the "Reprocessed" or "Science Quality" dataset on the NOAA ERDDAP site, often labeled as noaacwSlaDaily or similar.

Visit https://coastwatch.noaa.gov/cw_html/cwViewer.html
For documentation about the API check out https://coastwatch.noaa.gov/erddap/index.html
A list of all datasets is here: https://coastwatch.noaa.gov/erddap/info/index.html?page=1&itemsPerPage=2000 (Note not all datasets are publicly accessible)

In [13]:
# Recommended by Gemini; REVSIT THIS
# The dataset IDs are mapped to measures in last column of dataset list above
DATASET_ID = 'noaacwBLENDEDsshDaily' 

In [14]:
LAT_MIN = 24.0
LAT_MAX = 30.0
LON_MIN = -90.0
LON_MAX = -80.0

In [16]:
TARGET_DATE_STR = '2023-09-15' 

In [17]:
from datetime import datetime, timedelta

target_date = datetime.strptime(TARGET_DATE_STR, '%Y-%m-%d')

window_start = target_date + timedelta(days=5)
window_end = target_date - timedelta(days=5)

In [26]:
print(f"Fetching data from {DATASET_ID}...")
print(f"Time: {window_start} to {window_end}")
print(f"Box: Lat({LAT_MIN}), Lon({LON_MAX})")
    
erddap_obj.dataset_id = DATASET_ID
    
# Set constraints
erddap_obj.griddap_initialize()

erddap_obj.constraints['time>='] = window_start
erddap_obj.constraints['time<='] = window_end
erddap_obj.constraints['latitude>='] = LAT_MIN
erddap_obj.constraints['latitude<='] = LAT_MAX
erddap_obj.constraints['longitude>='] = LON_MIN
erddap_obj.constraints['longitude<='] = LON_MAX

Fetching data from noaacwBLENDEDsshDaily...
Time: 2023-09-20 00:00:00 to 2023-09-10 00:00:00
Box: Lat(24.0), Lon(-80.0)


Select the data to fetch below. "sla" is sea level anomaly (the deviation from mean sea surface hieght).

In [29]:
# Select variables to download (ssh/sla and coordinates)
erddap_obj.variables = ['sla']

In [32]:
data = erddap_obj.to_pandas()

In [33]:
data

,time (UTC),latitude (degrees_north),longitude (degrees_east),sla (m)
0,2023-09-10T00:00:00Z,24.125,-89.875,0.0157
1,2023-09-10T00:00:00Z,24.125,-89.625,0.0563
2,2023-09-10T00:00:00Z,24.125,-89.375,0.0960
3,2023-09-10T00:00:00Z,24.125,-89.125,0.1284
4,2023-09-10T00:00:00Z,24.125,-88.875,0.1520
...,...,...,...,...
11270,2023-09-20T00:00:00Z,30.125,-80.875,0.1559
11271,2023-09-20T00:00:00Z,30.125,-80.625,0.1408
11272,2023-09-20T00:00:00Z,30.125,-80.375,0.1310
11273,2023-09-20T00:00:00Z,30.125,-80.125,0.1145


In [34]:
data.to_csv('sealevel.csv')